In [ ]:
#shouldn't need to update - persists


#Step 1: Install necessary packages
#Selenium WebDriver is set up using chromium-browser in a headless mode, which allows you to run it in the background without a GUI.
!pip install selenium
!pip install webdriver-manager

# Step 2: Install Chromium and dependencies
!apt-get update # Update the system
!apt install -y chromium-browser  # Install Chromium

# if need to troubleshoot consider these
# !apt-get install --reinstall chromium-browser
# !which chromium-browser  # Check if it's installed at /usr/bin/chromium-browser

!apt-get install -y libnss3 libgdk-pixbuf2.0-0 libxss1
# libnss3 - Network Security Services - secure netw comms, SSL certs, encryption/auth etc
# libgdk-pixbuf2.0-0 libgdk-pixbuf2.0-0: GDK Pixbuf: This library is part of the GTK+ (GIMP Toolkit) and is responsible for loading and manipulating images in various formats (like PNG, JPEG, GIF).
# libasound2 - libasound2 Advanced Linux Sound Architecture? hy? handles audio content

In [ ]:
from google.colab import auth
auth.authenticate_user()

# import gspread
# from google.auth import default

# creds, _ = default()

# gc = gspread.authorize(creds)

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from datetime import datetime
from google.auth import default
from google.auth.transport.requests import Request
import gspread

# Step 1: Authenticate Google Colab using google-auth
creds, _ = default()

# Refresh the credentials if expired
if creds and creds.expired and creds.refresh_token:
    creds.refresh(Request())

# Step 2: Authorize gspread with the credentials
gc = gspread.authorize(creds)

# Step 3: Open Google Sheets by name
spreadsheet = gc.open('testposh')  # Replace with your sheet name
worksheet = spreadsheet.worksheet('MENS_JacketsCoats100_500_JustInSold')
# worksheet = spreadsheet.sheet1  # Choose the first sheet -- originall worked del

# Step 4: Setup Chrome WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in background
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Set the binary location to where Chromium is installed in Colab
options.binary_location = '/usr/bin/chromium-browser'

# Initialize the WebDriver using SeleniumManager (this auto handles the driver download)
driver = webdriver.Chrome(options=options)

# Step 5: Target URL
url = "https://poshmark.com/category/Men-Jackets_&_Coats?sort_by=added_desc&price%5B%5D=100-250&price%5B%5D=250-500&availability=sold_out"
driver.get(url)

# Step 6: Allow Time for JavaScript to Load
time.sleep(5)

# Step 7: Extract Brand Names and Prices
try:
    # Extract brands
    brand_elements = driver.find_elements(By.CLASS_NAME, "tile__title")
    brands = [brand.text for brand in brand_elements]

    # Extract prices (if available)
    price_elements = driver.find_elements(By.CLASS_NAME, "price")
    prices = [price.text for price in price_elements]

    # Fill missing prices with 'N/A' in case some listings don't have prices
    if len(prices) < len(brands):
        prices.extend(['N/A'] * (len(brands) - len(prices)))

    # Step 8: Ensure Data is Collected Correctly
    if brands:
        # Get today's date
        today_date = datetime.today().strftime('%m-%d-%Y')

        # Create DataFrame with Brand and Price
        data = {'Brand': brands, 'Price': prices, 'Date': [today_date] * len(brands)}

        # Create DataFrame
        df = pd.DataFrame(data)

        # Step 9: Convert DataFrame to list and append to Google Sheets
        data_to_append = df.values.tolist()
        worksheet.append_rows(data_to_append)  # Append data to Google Sheets

        # Print DataFrame for verification
        print(df)
    else:
        print("⚠️ No brands found.")
except Exception as e:
    print(f"⚠️ An error occurred: {e}")

# Step 10: Quit the WebDriver
driver.quit()



RefreshError: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7b829593c6d0>)